<a href="https://colab.research.google.com/github/cool250/nlp/blob/main/PyTorch_hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np

from sklearn.datasets import make_blobs
from matplotlib import pyplot

In [ ]:
samples = 5000
train_split = int(samples*0.8)

X,y = make_blobs(n_samples=samples, centers=2, n_features=64, cluster_std=10, random_state=2020)
y = y.reshape(-1,1)

X,y = torch.from_numpy(X), torch.from_numpy(y)
X,y = X.float(), y.float()

X_train, X_test = X[:train_split], X[train_split:]
y_train, y_test = y[:train_split], y[train_split:]

#Print shapes of each dataset
print("X_train.shape:",X_train.shape)
print("X_test.shape:",X_test.shape)
print("y_train.shape:",y_train.shape)
print("y_test.shape:",y_test.shape)
print("X.dtype",X.dtype)
print("y.dtype",y.dtype)

In [ ]:
class NeuralNetwork(nn.Module):

  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(64, 256)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(256,1024)
    self.relu2 = nn.ReLU()
    self.fc3 = nn.Linear(1024,1)
    self.final = nn.Sigmoid()

  def forward(self,x):
    op = self.fc1(x)
    op = self.relu1(op)
    op = self.fc2(op)
    op = self.relu2(op)
    op = self.fc3(op)
    y = self.final(op)
    return y

In [ ]:
def train_network(model, optimizer, loss_function, num_epochs, batch_size, X_train, y_train):
  model.train()
  loss_across_epochs = []
  for epoch in range(num_epochs):
    epoch_loss = 0.0

    for i in range(0, X_train.shape[0], batch_size):
      input_data = X_train[i:min(X_train.shape[0], i+batch_size)]
      labels = y_train[i:min(X_train.shape[0], i+batch_size)]

      optimizer.zero_grad()

      output = model(input_data)

      loss = loss_function(output, labels)

      loss.backward()

      optimizer.step()
      epoch_loss += loss.item()

    print("Epoch: {} - Loss {:4f}".format(epoch+1, epoch_loss))
    loss_across_epochs.extend([epoch_loss])

  return loss_across_epochs


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# Create an object of Neural Network class
model = NeuralNetwork()
model.to(device)

#Loss Function
loss_function = nn.BCELoss() # Binary Cross Entropy

# Define epochs, learning rate, and batch size
num_epochs = 10
batch_size = 16
learning_rate = 0.0001

# optimizer
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

loss = train_network(model, optimizer, loss_function, num_epochs, batch_size, X_train, y_train)

print("Loss - {}".format(loss))

In [ ]:
def test(model, loss_function, X_test, y_test):
  y_test_pred = model(X_test)
  a = np.where(y_test_pred > 0.5,1,0)
  return a